In [ ]:
install.packages(c('httr','rjson','RCurl','digest','foreach','iterators','devtools','curl','jsonlite','mime'))

In [ ]:
library(devtools)

In [ ]:
install_github("Azure/rAzureBatch")
install_github("Azure/doAzureParallel")

In [ ]:
install.packages("microbenchmark")
library(microbenchmark)
library(reshape2)
library(ggplot2)

In [ ]:
vmandelbrot <- function(xvec, y0, lim)
{
  mandelbrot <- function(x0,y0,lim)
  {
    x <- x0; y <- y0
    iter <- 0
    while (x^2 + y^2 < 4 && iter < lim)
    {
      xtemp <- x^2 - y^2 + x0
      y <- 2 * x * y + y0
      x <- xtemp
      iter <- iter + 1
    }
    iter
  }
 
  unlist(lapply(xvec, mandelbrot, y0=y0, lim=lim))
}

In [ ]:
localExecution <- function() {
  print("doParallel")
  library(doParallel)
  cl<-makeCluster(4)
  registerDoParallel(cl)
 
  x.in <- seq(-2, 1.5, length.out=1080)
  y.in <- seq(-1.5, 1.5, length.out=1080)
  m <- 1000
  mset <- foreach(i=y.in, .combine=rbind, .export = "vmandelbrot") %dopar% vmandelbrot(x.in, i, m)
}

In [ ]:
azureExecution <- function(pool_config) {
  print("doAzureParallel")
  library(doAzureParallel)
  pool <- doAzureParallel::makeCluster(pool_config)
  registerDoAzureParallel(pool)
 
  x.in <- seq(-2, 1.5, length.out=1080)
  y.in <- seq(-1.5, 1.5, length.out=1080)
  m <- 1000
  mset <- foreach(i=y.in, .combine=rbind, .options.azure = list(chunkSize=10), .export = "vmandelbrot") %dopar% vmandelbrot(x.in, i, m)
}

In [ ]:
op <- microbenchmark(
  #doParLocal=localExecution(),
  doParAzure=azureExecution(),
  #rxLocal=rxLocalExecution(),
  rxSpark=rxSparkExecution(),
  times=3L)

In [ ]:
print(op)

In [ ]:
plot(op)